In [1]:
import pandas as pd
import numpy as np
import re
import ast
import os 
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

# Set Pandas options to display full text for all columns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  # Show all columns


In [2]:
# Load the dataset from the './data' folder
people_path = './data/YC F24 Founder People.csv'

# Read data into pandas DataFrame
people_data = pd.read_csv(people_path)

# Relevant columns to keep
relevant_columns = [
    'full_name', 'title', 'location', 'company', 'company_linkedin', 
    'company_description', 'sub_title', 'summary', 'current_job_description', 
    'person_industry', 'skills', 'education', 'work_history'
]

# Filter the relevant columns
people_filtered = people_data[relevant_columns]

# Display basic information about the dataset
print("Dataset Loaded:")
display(people_filtered.head())


Dataset Loaded:


full_name               title  \
0   Edward Aryee    Co-Founder & CTO   
1   Victor Cheng  Co-Founder and CEO   
2  Daryl Budiman    Co-Founder & CEO   
3   Finn Mallery          Co-Founder   
4      Arvind V.  Co-Founder and CEO   

                                                              location  \
0  San Francisco Bay, San Francisco Bay Area, United States of America   
1                  San Francisco, California, United States of America   
2  San Francisco Bay, San Francisco Bay Area, United States of America   
3                      Palo Alto, California, United States of America   
4                  San Francisco, California, United States of America   

                   company                             company_linkedin  \
0          SRE.ai (YC F24)  https://www.linkedin.com/company/104394435/   
1          vly.ai (YC F24)   https://www.linkedin.com/company/99356232/   
2         Andoria (YC F24)  https://www.linkedin.com/company/102953213/   
3  Origami Agents (YC F24)   https://www.linkedin.com/company/99440945/   
4          Fresco (YC F24)  https://www.linkedin.com/company/104799979/   

  company_description  \
0                 NaN   
1                 NaN   
2                 NaN   
3                 NaN   
4                 NaN   

                                                   sub_title  \
0            Co-Founder & CTO at SRE.ai (YC F24) | ex-Google   
1                               co-founder @ vly.ai (YC F24)   
2  Co-Founder at Andoria (YC F24) | Prev. MultiOn & Stanford   
3                       Co-Founder @ Origami Agents (YC F24)   
4                Building Fresco (YC F24) | Acquired Founder   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            summary  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               NaN   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           cooking   
2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Imagine 

In [3]:
# Helper function to extract the "name" field using a simple regex
def extract_names_simple(data_str):
    try:
        # Find all occurrences of "name": "value"
        names = re.findall(r'"name":\s?"(.*?)"', data_str)
        return ", ".join(names) if names else None
    except Exception as e:
        return None  # Return None if parsing fails

# Helper function to convert the skills column into a readable string
def process_skills(skills_str):
    try:
        # Convert the string representation of a list into an actual list
        skills_list = ast.literal_eval(skills_str)
        if isinstance(skills_list, list):
            return ", ".join(skills_list)  # Join the list into a string
        return skills_str  # If not a list, return the original string
    except (ValueError, SyntaxError):
        return None  # Return None if parsing fails

# Apply the updated function to the education, work history, and skills columns
def preprocess_columns_simple(df):
    df['education'] = df['education'].apply(extract_names_simple)
    df['work_history'] = df['work_history'].apply(extract_names_simple)
    df['skills'] = df['skills'].apply(process_skills)
    return df

# Preprocess the dataset
people_preprocessed = preprocess_columns_simple(people_filtered.copy())


In [4]:
# Helper function to generate a descriptive paragraph for each row
def generate_paragraph(row):
    # Start the paragraph with basic information
    full_name = row.get('full_name', None)
    title = row.get('title', None)
    company = row.get('company', None)
    location = row.get('location', None)
    company_description = row.get('company_description', None)
    sub_title = row.get('sub_title', None)
    summary = row.get('summary', None)
    current_job_description = row.get('current_job_description', None)
    skills = row.get('skills', None)
    education = row.get('education', None)
    work_history = row.get('work_history', None)

    # Build the descriptive paragraph dynamically
    sentences = []

    if full_name and title and company:
        sentences.append(f"{full_name}, currently working as {title} at {company}.")
    if location:
        sentences[-1] += f" They are based in {location}."
    if company_description:
        sentences.append(f"{company} specializes in {company_description}.")
    if sub_title:
        sentences.append(f"Sub-title: {sub_title}.")
    if summary:
        sentences.append(f"Summary: {summary}.")
    if current_job_description:
        sentences.append(f"Current Job Description: {current_job_description}.")
    if skills:
        sentences.append(f"Their skills include: {skills}.")
    if education:
        sentences.append(f"Their educational background includes: {education}.")
    if work_history:
        sentences.append(f"They have worked in roles such as: {work_history}.")

    # Combine sentences into a single paragraph
    return " ".join(sentences)

# Apply the function to generate paragraphs
people_preprocessed['descriptive_paragraph'] = people_preprocessed.apply(generate_paragraph, axis=1)

# Display a few sample paragraphs
print("Sample Descriptive Paragraphs:")
display(people_preprocessed[['full_name', 'descriptive_paragraph']].head())


Sample Descriptive Paragraphs:


,full_name,descriptive_paragraph
0,Edward Aryee,"Edward Aryee, currently working as Co-Founder & CTO at SRE.ai (YC F24). They are based in San Francisco Bay, San Francisco Bay Area, United States of America. SRE.ai (YC F24) specializes in nan. Sub-title: Co-Founder & CTO at SRE.ai (YC F24) | ex-Google. Summary: nan. Current Job Description: An AI-powered automation platform for Salesforce development teams.. Their skills include: Programming, Python, Java, JavaScript, Matlab, Computer Science, Public Speaking. Their educational background includes: Drexel University. They have worked in roles such as: SRE.ai (YC F24), Black Angel Group (BAG), Google, Microsoft, StratIS IoT, Web Presence in China, BuLogics, Virtual Pantry, Columbia University."
1,Victor Cheng,"Victor Cheng, currently working as Co-Founder and CEO at vly.ai (YC F24). They are based in San Francisco, California, United States of America. vly.ai (YC F24) specializes in nan. Sub-title: co-founder @ vly.ai (YC F24). Summary: cooking. Current Job Description: The easiest way to build custom full-stack software without code. Backed by Y Combinator. Their skills include: Startups, TypeScript, Git, Express.js, MongoDB, Full-Stack Development, Node.js, Competitive Programming, Next.js, Software Development, React.js, C++, JavaScript, C#, Java, Unity3D, Business, Mathematics, Robotics, Mobile Application Development, Android Studio. Their educational background includes: University of Washington, Skyline High School, Y Combinator. They have worked in roles such as: vly.ai (YC F24), Hatch Coding, Minute Land, Quantum Labs, Autodesk, FRC Team 2976, Spartabots, Self-employed."
2,Daryl Budiman,"Daryl Budiman, currently working as Co-Founder & CEO at Andoria (YC F24). They are based in San Francisco Bay, San Francisco Bay Area, United States of America. Andoria (YC F24) specializes in nan. Sub-title: Co-Founder at Andoria (YC F24) | Prev. MultiOn & Stanford. Summary: Imagine if you didn't have to learn any piece of software anymore.. Current Job Description: Backed by YC, among other great investors.. Their skills include: Customer Onboarding, Customer Success, SwiftUI, C++, React.js, Microsoft PowerPoint, HTML, CSS, JavaScript, Python, Presentation Skills, Public Speaking, Adobe Photoshop, Research, ArcGIS, Figma, Leadership, User Interface Design, Computer-Aided Design (CAD), Autodesk Fusion 360. Their educational background includes: Stanford University, Y Combinator. They have worked in roles such as: Andoria (YC F24), MultiOn, Amazon, The Marriage Pact, Evernote, Matcha, Stanford University School of Medicine."
3,Finn Mallery,"Finn Mallery, currently working as Co-Founder at Origami Agents (YC F24). They are based in Palo Alto, California, United States of America. Origami Agents (YC F24) specializes in nan. Sub-title: Co-Founder @ Origami Agents (YC F24). Summary: nan. Current Job Description: nan. Their educational background includes: Stanford University, Stanford University School of Engineering, Z Fellows, Canyon Crest Academy. They have worked in roles such as: Origami Agents (YC F24), Fizz, StartX., Stanford Marketing."
4,Arvind V.,"Arvind V., currently working as Co-Founder and CEO at Fresco (YC F24). They are based in San Francisco, California, United States of America. Fresco (YC F24) specializes in nan. Sub-title: Building Fresco (YC F24) | Acquired Founder. Summary: I’m currently building Fresco (YC F24) an AI copilot for construction superintendents. \n\nPreviously, I was Chief of Staff and Director of Partnerships at January AI, a precision medicine company based in the Bay Area. I have my BA and MS from Brown, and I've also worked at NASA (SynBio), Human Capital (VC), and Vault Health (PM).\n\nI enjoyed an exit from my political tech startup, DownBallot Solutions, in 2022. I also make investments in early-stage technology companies, with a focus on consumer software and digital health. I'm always happy to chat about science or entrepreneurship!. Current Job

In [6]:
# Load the new dataset
personal_data_path = './data/Sudarshan Sridharan People.csv'
personal_data = pd.read_csv(personal_data_path)

# Preprocessing functions reused from the previous notebook
# (No need to redefine extract_names_simple and process_skills)

# Helper function to process the new file into a descriptive paragraph
def generate_advertising_paragraph(row):
    # Basic information
    full_name = row.get('full_name', None)
    title = row.get('title', None)
    company = row.get('company', None)
    location = row.get('location', None)
    company_description = row.get('company_description', None)
    work_history = row.get('work_history', None)
    education = row.get('education', None)
    skills = row.get('skills', None)

    # Build the paragraph dynamically
    sentences = []
    
    if full_name and title and company:
        sentences.append(f"{full_name} is the {title} at {company}.")
    if location:
        sentences[-1] += f" They are based in {location}."
    if company_description:
        sentences.append(f"{company} focuses on: {company_description}.")
    if skills:
        sentences.append(f"Key skills include: {skills}.")
    if education:
        sentences.append(f"Educational background includes: {education}.")
    if work_history:
        sentences.append(f"Past work includes roles such as: {work_history}.")

    # Combine and clean up
    return " ".join(sentences)

# Preprocess the new dataset
personal_data['education'] = personal_data['education'].apply(extract_names_simple)
personal_data['work_history'] = personal_data['work_history'].apply(extract_names_simple)
personal_data['skills'] = personal_data['skills'].apply(process_skills)

# Generate the descriptive paragraph
personal_data['advertising_paragraph'] = personal_data.apply(generate_advertising_paragraph, axis=1)

# Display the output
print("Sample Advertising Paragraphs:")
display(personal_data[['full_name', 'advertising_paragraph']].head())


Sample Advertising Paragraphs:


,full_name,advertising_paragraph
0,Sudarshan Sridharan,"Sudarshan Sridharan is the Founder at Pipeline. They are based in San Francisco Bay, San Francisco Bay Area, United States of America. Pipeline focuses on: Get more leads, close more deals. Pipeline sets-up and manages the outbound sales stack for early stage B2B startups. Automate your sales.... Key skills include: Investing. Educational background includes: Clemson University. Past work includes roles such as: Pipeline, Fion Technologies, SaveMAPS, Second Reality Interactive, INC., Peak 15 Capital."


In [10]:
# Ensure 'people_preprocessed' and 'new_people_data' DataFrames exist from previous steps

# Extract the single paragraph from the new people's data
advertising_paragraph = personal_data['advertising_paragraph'].iloc[0]  # Assuming only one row

# Create a DataFrame with pairs of paragraphs
paired_paragraphs = people_preprocessed[['full_name', 'descriptive_paragraph']].copy()
paired_paragraphs['advertising_paragraph'] = advertising_paragraph  # Add the single advertising paragraph to all rows

# Display the paired paragraphs
print("Paired Paragraphs (Personal Profile and Advertising Profile):")
display(paired_paragraphs.head())


Paired Paragraphs (Personal Profile and Advertising Profile):


,full_name,descriptive_paragraph,advertising_paragraph
0,Edward Aryee,"Edward Aryee, currently working as Co-Founder & CTO at SRE.ai (YC F24). They are based in San Francisco Bay, San Francisco Bay Area, United States of America. SRE.ai (YC F24) specializes in nan. Sub-title: Co-Founder & CTO at SRE.ai (YC F24) | ex-Google. Summary: nan. Current Job Description: An AI-powered automation platform for Salesforce development teams.. Their skills include: Programming, Python, Java, JavaScript, Matlab, Computer Science, Public Speaking. Their educational background includes: Drexel University. They have worked in roles such as: SRE.ai (YC F24), Black Angel Group (BAG), Google, Microsoft, StratIS IoT, Web Presence in China, BuLogics, Virtual Pantry, Columbia University.","Sudarshan Sridharan is the Founder at Pipeline. They are based in San Francisco Bay, San Francisco Bay Area, United States of America. Pipeline focuses on: Get more leads, close more deals. Pipeline sets-up and manages the outbound sales stack for early stage B2B startups. Automate your sales.... Key skills include: Investing. Educational background includes: Clemson University. Past work includes roles such as: Pipeline, Fion Technologies, SaveMAPS, Second Reality Interactive, INC., Peak 15 Capital."
1,Victor Cheng,"Victor Cheng, currently working as Co-Founder and CEO at vly.ai (YC F24). They are based in San Francisco, California, United States of America. vly.ai (YC F24) specializes in nan. Sub-title: co-founder @ vly.ai (YC F24). Summary: cooking. Current Job Description: The easiest way to build custom full-stack software without code. Backed by Y Combinator. Their skills include: Startups, TypeScript, Git, Express.js, MongoDB, Full-Stack Development, Node.js, Competitive Programming, Next.js, Software Development, React.js, C++, JavaScript, C#, Java, Unity3D, Business, Mathematics, Robotics, Mobile Application Development, Android Studio. Their educational background includes: University of Washington, Skyline High School, Y Combinator. They have worked in roles such as: vly.ai (YC F24), Hatch Coding, Minute Land, Quantum Labs, Autodesk, FRC Team 2976, Spartabots, Self-employed.","Sudarshan Sridharan is the Founder at Pipeline. They are based in San Francisco Bay, San Francisco Bay Area, United States of America. Pipeline focuses on: Get more leads, close more deals. Pipeline sets-up and manages the outbound sales stack for early stage B2B startups. Automate your sales.... Key skills include: Investing. Educational background includes: Clemson University. Past work includes roles such as: Pipeline, Fion Technologies, SaveMAPS, Second Reality Interactive, INC., Peak 15 Capital."
2,Daryl Budiman,"Daryl Budiman, currently working as Co-Founder & CEO at Andoria (YC F24). They are based in San Francisco Bay, San Francisco Bay Area, United States of America. Andoria (YC F24) specializes in nan. Sub-title: Co-Founder at Andoria (YC F24) | Prev. MultiOn & Stanford. Summary: Imagine if you didn't have to learn any piece of software anymore.. Current Job Description: Backed by YC, among other great investors.. Their skills include: Customer Onboarding, Customer Success, SwiftUI, C++, React.js, Microsoft PowerPoint, HTML, CSS, JavaScript, Python, Presentation Skills, Public Speaking, Adobe Photoshop, Research, ArcGIS, Figma, Leadership, User Interface Design, Computer-Aided Design (CAD), Autodesk Fusion 360. Their educational background includes: Stanford University, Y Combinator. They have worked in roles such as: Andoria (YC F24), MultiOn, Amazon, The Marriage Pact, Evernote, Matcha, Stanford University School of Medicine.","Sudarshan Sridharan is the Founder at Pipeline. They are based in San Francisco Bay, San Francisco Bay Area, United States of America. Pipeline focuses on: Get more leads, close more deals. Pipeline sets-up and manages the outbound sales stack for early stage B2B startups. Automate your sales.... Key skills include: Investing. Educational backgro

In [8]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)
print(message.content)


KeyboardInterrupt: 